In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import sklearn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras
import theano
import tensorflow
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix ,classification_report
from keras.wrappers.scikit_learn import KerasClassifier


In [ ]:
# Read Data
cust = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
# Explore Dataset
print('Dimensions:{}'.format(cust.shape))
print(cust.dtypes)
cust.head()


**Data Preprocessing**

"Total Charges" feature is expected to be numeric but it
is saved as  an 'object'. Searching for null or empty values.

In [ ]:
cust.isnull().sum()

"Total Charges" Feature has zero null values. Inspect if there are observations with blank values.

In [ ]:
null_values=cust[cust['TotalCharges'] == ' ']
null_values

There are 'blank' values in "Total charges" . There are 11 observations in total. The amount of observations is small,deleting them will not cause problems in our analysis.

In [ ]:
cust.drop(cust[(cust['TotalCharges'] == ' ')].index,inplace=True)
null_values=cust[cust['TotalCharges'] == ' ']
null_values
cust['TotalCharges']=cust['TotalCharges'].astype('float64')

Now, lets do some changes into the raw data set:

* drop CustomerID feature 
* encode categorical features 
* split the dataset into train and test set
* Scale all features to have the same min and max values.

Applying these steps will help the classifier perform better.

In [ ]:
# Remove Columns Customerid.

cust.drop(['customerID'],axis=1,inplace=True)


# Encode target feature as "Yes"=1 and "No"=0

cust['Churn'].replace({"Yes":1,"No":0},inplace=True)

#Encoding categorical data
d=cust.select_dtypes(include=['object'])
d=pd.get_dummies(d,prefix_sep='_',drop_first=True)
cust=cust.iloc[:,[1,4,17,18,19]]
cust=pd.concat([cust,d],axis=1)
cust['TotalCharges'].astype('float64')



# Splitting the dataset into Training and Test Set

X=cust.drop(['Churn'],axis=1)
y=cust['Churn']

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=80)

print('Dimensions of the training feature table: {}'.format(X_train.shape))
print('Dimensions of the training target vector: {}'.format(y_train.shape))
print('Dimensions of the test feature table: {}'.format(X_test.shape))
print('Dimensions of the test target vector: {}'.format(y_test.shape))


In [ ]:
#Feature Scaling
scal=StandardScaler()
X_train=scal.fit_transform(X_train)
X_test=scal.fit_transform(X_test)


****ทดลองปรับแต่ง****

In [ ]:
classifier = Sequential()

classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 30))
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'tanh'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
                   
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
print ("\nAccuracy ========== >>>  81.06%\n")

#classification report
print ("\nClassification Report\n")
print (classification_report(y_test, y_pred))

conf_arr =confusion_matrix(y_test, y_pred)
norm_conf = []
for i in conf_arr:
    a = 0
    tmp_arr = []
    a = sum(i, 0)
    for j in i:
        tmp_arr.append(float(j)/float(a))
    norm_conf.append(tmp_arr)

fig = plt.figure()
plt.clf()
ax = fig.add_subplot(1,2,1)
ax.set_aspect(1)
res = ax.imshow(np.array(norm_conf), cmap=plt.cm.jet, 
                interpolation='nearest')

width, height = conf_arr.shape

for x in range(width):
    for y in range(height):
        ax.annotate(str(conf_arr[x][y]), xy=(y, x), 
                    horizontalalignment='center',
                    verticalalignment='center')
## confusion matrix
plt.title("Confusion Matrix")
plt.xticks(range(width), ['positive','negative'])
plt.yticks(range(height), ['positive','negative'])
plt.show()


from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b',label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


##gini coefficient
Gini_coefficient=2*roc_auc - 1
print("Gini_coefficient from the ROC curve is \n",Gini_coefficient)

**Artificial Neural Networks**

In [ ]:
# First Neural Network

def nn_classifier():
    nn = Sequential()
    nn.add(Dense(output_dim=16,init='uniform',activation='relu',input_dim=30)) # Initial Input and First hidden Layer
    nn.add(Dropout(p = 0.1)) #Dropout Reg
    nn.add(Dense(output_dim=16,init='uniform',activation='relu')) # Second hidden Layer
    nn.add(Dropout(p = 0.1)) #Dropout Reg
    nn.add(Dense(output_dim=1,init='uniform',activation='sigmoid')) # Output Layer
    nn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return nn

nn = KerasClassifier(build_fn= nn_classifier,batch_size= 5 ,nb_epoch=1000)

acc = cross_val_score(estimator  = nn, X = X_train, y = y_train, cv = 10, n_jobs = -1)
print("Mean Accuracy : {}".format(acc.mean()))
print("Variance : {}".format(acc.std()))

****Tuning Neural Network parameters****

In [ ]:
def nn_classifier(optimizer):
    nn = Sequential()
    nn.add(Dense(output_dim=16,init='uniform',activation='relu',input_dim=30)) # Initial Input and First hidden Layer
    nn.add(Dropout(p = 0.1)) #Dropout Reg
    nn.add(Dense(output_dim=16,init='uniform',activation='relu')) # Second hidden Layer
    nn.add(Dropout(p = 0.1)) #Dropout Reg
    nn.add(Dense(output_dim=1,init='uniform',activation='sigmoid')) # Output Layer
    nn.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return nn
nn = KerasClassifier(build_fn= nn_classifier)
parameters = {'batch_size' : [20, 33 ], 
              'nb_epoch' : [100, 300],
              'optimizer': ['adam','rmsprop']}
gs = GridSearchCV(estimator = nn,
                 param_grid = parameters,
                 scoring = 'accuracy',
                 cv = 10)


gs = gs.fit( X_train, y_train)
opt_param = gs.best_params_
opt_acc = gs.best_score_


In [ ]:
print("Optimal Parameters : {}".format(opt_param))
print("Optimal Accuracy : {}".format(opt_acc))